In [7]:
#importar as bibliotecas utilizadas 
import sys
assert sys.version_info >= (3 , 5)

import sklearn 
assert sklearn.__version__ >= "0.20"

try: 
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0" 

%load_ext tensorboard

import numpy as np
import os

np.random.seed(42)

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [9]:
#vamos carregar os datasets
(X_train_full, y_train_full),(X_test, y_test) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 1s 0us/step


In [11]:
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandálias ou camisas
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # índices das classes 7, 8, 9 deveriam ser alterados para 5, 6, 7
    y_B= (y[y_5_or_6] == 6).astype(np.float32) # classificação binária: é uma camisa? (classe 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))
(X_train_A, y_train_A),(X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A),(X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
    
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [12]:
X_train_A.shape

(43986, 28, 28)

In [13]:
X_train_B.shape

(200, 28, 28)

In [14]:
tf.random.set_seed(42)
np.random.seed(42)

In [17]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28,28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [19]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])